In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

from datetime import datetime
from F1_process_functions import func_linear_interp_dates

## Merging all MSOA data 

In this notebook, we merge the transient MSOA data to use in forecasting.

This is the final version. Other versions will be removed.

Features:
1. Income
2. Houseprice
3. Electricity consumption
4. Index of multiple deprivation decile
5. Rural-urban classification (cat)
6. EV count (transient) <- variable to predict
7. Public EV charger count (transient)
8. Private EV charger count (transient)
9. Solar panel count (transient)
10. Average EV price (transient)


Time frame: Jan 2014 - April 2021 monthly

Keeping MSOA codes in England and Wales for which there are MOT centres (N_m = 5740)

## Steady variables

Load the steady features .csv and retain the steady features only. This data has been processed and only contains the MSOA regions with MOT centres.

### 1.-5. Income, Houseprice, Electricity Consumption, IMD, RUC

In [2]:
# load income and just keep most recent as we will not use this as a time-dependent variable
df_steady = pd.read_csv('data/msoa_today_steady.csv')
df_steady = df_steady[['MSOA11CD','total_annual_income_2019','houseprice_avg_2019',
                       'elec_consumption_2019', 'IMDRank','IMDDecil','RUC11']]
print(df_steady.shape)
df_steady.head()

(5740, 7)


,MSOA11CD,total_annual_income_2019,houseprice_avg_2019,elec_consumption_2019,IMDRank,IMDDecil,RUC11
0,E02004297,40650.0,129990.000000,15387.466659,20450.20,6.0,Rural town and fringe
1,E02004290,43260.0,135343.750000,12941.948564,21972.50,8.0,Urban major conurbation
2,E02004298,40635.0,136866.666667,19825.929444,18949.50,6.5,Urban city and town
3,E02004299,34450.0,109200.000000,14636.314372,10635.60,4.0,Urban city and town
4,E02004291,34730.0,95375.000000,11049.599268,7844.25,2.5,Urban major conurbation


## Transient variables

We will keep 2014-01 onwards.


In [3]:
ordered_dates = pd.date_range("2014-01-01", periods=88, freq="MS")

### 6. EV Count (target variable)

First select MSOA that are consistent with steady features. Then select the MSOA in the remaining dataframe as the MSOA regions to keep throughout analysis.

In [4]:
df1 = pd.read_csv('data/msoa_ev_count_dated.csv')
df1.set_index('msoa11cd', inplace=True)
for col in df1.columns:
    df1.rename(columns={col:col[:7]},inplace=True)
#df1 = df1[['index','2021-04']]
#df1.rename(columns={'index':'MSOA11CD', '2021-04':'ev_count_2021'}, inplace=True)
df1 = df1[df1.columns.sort_values()]
df1 = df1[df1.columns[-88:]]
df1.head()

,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,...,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04
msoa11cd,,,,,,,,,,,,,,,,,,,,,
E02000002,2,2,2,2,2,2,2,2,2,2,...,6,6,6,6,6,6,6,6,6,6
E02000003,3,4,4,4,4,4,4,4,4,4,...,42,42,42,42,42,42,42,42,42,42
E02000005,0,0,0,0,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
E02000009,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
E02000010,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [5]:
for col_name in df1.columns:
    df1.rename(columns={col_name: pd.to_datetime(col_name + '-01')},inplace=True)

In [6]:
print(df_steady.shape)
print(df1.shape)

(5740, 7)
(4922, 88)


In [7]:
#Need to pad MSOAs with zero EVs
# drop msoa that aren't in steady
df1.reset_index(inplace=True)
# Pad missing MSOA regions
missing_msoa = list(set(df_steady.MSOA11CD.unique()) - set(df1.msoa11cd.unique()))

#df1.set_index(['msoa11cd'],inplace=True)
# store the dates
columns_names = df1.columns.values

# for each msoa add a row of zeros
df_missing_msoa = pd.DataFrame(columns=columns_names)
for i in range(len(missing_msoa)):
    df_missing_msoa = df_missing_msoa.append(pd.Series({"msoa11cd": missing_msoa[i],
                         columns_names[1]: 0}),ignore_index=True)
df_missing_msoa.fillna(0,inplace=True)

df1= df1.append(df_missing_msoa)
df1.reset_index(drop=True)

df1 = df1[df1['msoa11cd'].isin(df_steady.MSOA11CD.unique())]

print(df1.shape)
print(df1.msoa11cd.nunique())

(5740, 89)
5740


In [8]:
# stack the data
df1.set_index('msoa11cd',inplace=True)
df1.stack().isna().sum()


0

In [9]:
df1.columns[-1]

Timestamp('2021-04-01 00:00:00')

In [10]:
df_full = pd.DataFrame(df1.stack(),columns=['ev_count'])
df_full

ev_count
msoa11cd                      
E02000002 2014-01-01         2
          2014-02-01         2
          2014-03-01         2
          2014-04-01         2
          2014-05-01         2
...                        ...
E02004148 2020-12-01         0
          2021-01-01         0
          2021-02-01         0
          2021-03-01         0
          2021-04-01         0

[505120 rows x 1 columns]

In [11]:
msoa_to_keep = df_full.index.droplevel(1).unique()

## Charger data

7. Private charger data available from 2015 annually.
8. Public charger data available from 10-2019 quarterly.

In [12]:
df_private_chargers = pd.read_csv('data/private_chargers_msoa_timeseries.csv',index_col=0)
df_public_chargers = pd.read_csv('data/public_chargers_msoa_timeseries.csv',index_col=0)
df_private_chargers.drop(df_private_chargers[~df_private_chargers['MSOA11CD'].isin(np.array(msoa_to_keep))].index,inplace=True)
df_public_chargers.drop(df_public_chargers[~df_public_chargers['MSOA11CD'].isin(np.array(msoa_to_keep))].index,inplace=True)

### 7. Public chargers

Drop LSOA data, and extrapolate backwards

In [13]:
df_public_chargers = df_public_chargers[['MSOA11CD','10-2019_msoa', '01-2020_msoa', '04-2020_msoa', '07-2020_msoa',
       '10-2020_msoa', '01-2021_msoa', '04-2021_msoa']]
df_public_chargers.rename(columns={'10-2019_msoa':pd.to_datetime('2019-10-01'),
                                   '01-2020_msoa':pd.to_datetime('2020-01-01'),
                                   '04-2020_msoa':pd.to_datetime('2020-04-01'),
                                   '07-2020_msoa':pd.to_datetime('2020-07-01'),
                                   '10-2020_msoa':pd.to_datetime('2020-10-01'), 
                                   '01-2021_msoa':pd.to_datetime('2021-01-01'),
                                   '04-2021_msoa':pd.to_datetime('2021-04-01')},inplace=True)
df_public_chargers.set_index('MSOA11CD',inplace=True)

In [14]:
df_public_chargers = func_linear_interp_dates(df_public_chargers, ordered_dates)
df_public_chargers.shape

(5740, 88)

### 8. Private chargers

In [15]:
df_private_chargers = df_private_chargers[['MSOA11CD', '2015_msoa', '2016_msoa', '2017_msoa',
       '2018_msoa', '2019_msoa', '2020_msoa', '2021_msoa']]
df_private_chargers.set_index('MSOA11CD',inplace=True)
df_private_chargers.rename(columns={'2015_msoa': pd.to_datetime('2015-01-01'),
                                    '2016_msoa':pd.to_datetime('2016-01-01'),
                                    '2017_msoa':pd.to_datetime('2017-01-01'),
                                    '2018_msoa':pd.to_datetime('2018-01-01'),
                                    '2019_msoa':pd.to_datetime('2019-01-01'),
                                    '2020_msoa':pd.to_datetime('2020-01-01'),
                                    '2021_msoa':pd.to_datetime('2021-01-01')},inplace=True)

In [16]:
df_private_chargers = func_linear_interp_dates(df_private_chargers, ordered_dates)
df_private_chargers.shape

(5740, 88)

### 7. (T) PV count

In [17]:
df_pv = pd.read_csv('data/pv_msoa_fit_timeseries_2014.csv',index_col=0)
df_pv.reset_index(level=0, inplace=True)
df_pv.rename(columns={'MLSOA Code':'MSOA11CD'},inplace=True)
print(df_pv.shape)
df_pv.head()

(8443, 67)


,MSOA11CD,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,...,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2020-01,2020-02,2020-03
0,E02000001,0,0,0,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
1,E02000002,0,1,1,1,1,1,1,1,1,...,18,18,18,18,19,19,19,19,19,19
2,E02000003,0,1,1,2,2,2,2,2,2,...,15,15,15,15,16,16,17,17,17,17
3,E02000004,0,0,1,1,1,1,3,3,3,...,11,11,11,11,11,11,11,11,11,11
4,E02000005,1,0,0,0,0,1,1,1,3,...,81,81,81,82,82,82,82,82,82,82


In [18]:
# Retain only MSOAs with MOT centres
df_pv.drop(df_pv[~df_pv['MSOA11CD'].isin(np.array(df_steady.MSOA11CD.unique()))].index,inplace=True)
df_pv.shape

(5734, 67)

In [19]:
# Pad missing MSOA regions
missing_msoa = list(set(df_steady.MSOA11CD.unique()) - set(df_pv.MSOA11CD.unique()))

#df1.set_index(['msoa11cd'],inplace=True)
# store the dates
columns_names = df_pv.columns.values

# for each msoa add a row of zeros
df_missing_msoa = pd.DataFrame(columns=columns_names)
for i in range(len(missing_msoa)):
    df_missing_msoa = df_missing_msoa.append(pd.Series({"MSOA11CD": missing_msoa[i],
                         columns_names[1]: 0}),ignore_index=True)
df_missing_msoa.fillna(0,inplace=True)

df_pv= df_pv.append(df_missing_msoa)
df_pv.reset_index(drop=True)

df_pv = df_pv[df_pv['MSOA11CD'].isin(df_steady.MSOA11CD.unique())]

print(df_pv.shape)
print(df_pv.MSOA11CD.nunique())

(5740, 67)
5740


In [20]:
# make columns timestamps, and extrapolate forward
df_pv.set_index(['MSOA11CD'],inplace=True)

In [21]:
for col_name in df_pv.columns:
    df_pv.rename(columns={col_name: pd.to_datetime(col_name + '-01')},inplace=True)

In [22]:
missing_dates = list(set(ordered_dates)-set(df1.columns))
len(missing_dates)

0

In [23]:
# Extrapolate forward missing current dates
df_pv = func_linear_interp_dates(df_pv, ordered_dates)

In [24]:
df_pv.shape

(5740, 88)

### 10. EV price

In [25]:
data_prices = {pd.to_datetime('2015-03-01'): [25278.29],
               pd.to_datetime('2015-09-01'): [25358.64],
               pd.to_datetime('2016-03-01'): [24005.53], 
               pd.to_datetime('2016-09-01'): [24232.42],
               pd.to_datetime('2017-03-01'): [29987.22],
               pd.to_datetime('2017-09-01'): [30312.54], 
               pd.to_datetime('2018-03-01'): [30201.62], 
               pd.to_datetime('2018-09-01'): [30570.0], 
               pd.to_datetime('2019-03-01'): [35588.92],
               pd.to_datetime('2019-09-01'): [35211.67],
               pd.to_datetime('2020-03-01'): [32866.25], 
               pd.to_datetime('2020-09-01'): [33102.64], 
               pd.to_datetime('2021-03-01'): [45427.67]}

df_ev_prices =  pd.DataFrame(data=data_prices)
df_ev_prices

,2015-03-01,2015-09-01,2016-03-01,2016-09-01,2017-03-01,2017-09-01,2018-03-01,2018-09-01,2019-03-01,2019-09-01,2020-03-01,2020-09-01,2021-03-01
0,25278.29,25358.64,24005.53,24232.42,29987.22,30312.54,30201.62,30570.0,35588.92,35211.67,32866.25,33102.64,45427.67


In [26]:
df_ev_prices = func_linear_interp_dates(df_ev_prices, ordered_dates)
print(df_ev_prices.shape)

(1, 88)


## Stack all the variables

In [27]:
# 1. steady
df_steady.reset_index(inplace=True)
df_steady.rename(columns={'msoa11cd':'MSOA11CD'},inplace=True)
df_steady.set_index('MSOA11CD',inplace=True)
df_steady.head()

,index,total_annual_income_2019,houseprice_avg_2019,elec_consumption_2019,IMDRank,IMDDecil,RUC11
MSOA11CD,,,,,,,
E02004297,0,40650.0,129990.000000,15387.466659,20450.20,6.0,Rural town and fringe
E02004290,1,43260.0,135343.750000,12941.948564,21972.50,8.0,Urban major conurbation
E02004298,2,40635.0,136866.666667,19825.929444,18949.50,6.5,Urban city and town
E02004299,3,34450.0,109200.000000,14636.314372,10635.60,4.0,Urban city and town
E02004291,4,34730.0,95375.000000,11049.599268,7844.25,2.5,Urban major conurbation


In [28]:
msoa_index =  df_full.index.get_level_values('msoa11cd')
date_index = df_full.index.get_level_values(1)

In [29]:
# steady cars
df_full = pd.concat([df_full, df_steady.loc[msoa_index].set_index(df_full.index)], axis=1)

In [30]:
# 2. pv count
df_full = pd.concat([df_full, pd.DataFrame(df_pv.stack(),columns=['pv_count'])], axis=1)

In [31]:
# 3. public charger count
df_full = pd.concat([df_full, pd.DataFrame(df_public_chargers.stack(),columns=['public_charger_count'])], axis=1)

In [32]:
# 4. private charger count
df_full = pd.concat([df_full, pd.DataFrame(df_private_chargers.stack(),columns=['private_charger_count'])], axis=1)

In [33]:
# 5. ev price 
df_ev_prices_to_concat = df_ev_prices.transpose().rename(columns={'0':'ev_price'})
df_ev_prices_to_concat.columns = ["ev_price"]

#6. monthly count
# We also add a monthly count to capture temporal info
df_ev_prices_to_concat.insert(0, 'n_months', range(0, len(df_ev_prices_to_concat)))

In [34]:
df_ev_prices_to_concat = df_ev_prices_to_concat.loc[date_index].set_index(df_full.index)

In [35]:
df_full = pd.concat([df_full, df_ev_prices_to_concat], axis=1)

In [36]:
df_full.isna().sum().sum()

0

In [37]:
df_full.RUC11.nunique()

8

In [38]:
df_full.index.set_names(['MSOA11CD', 'date'], inplace=True)

In [39]:
df_full.head()

ev_count  index  total_annual_income_2019  \
MSOA11CD  date                                                    
E02000002 2014-01-01         2   3434                   40170.0   
          2014-02-01         2   3434                   40170.0   
          2014-03-01         2   3434                   40170.0   
          2014-04-01         2   3434                   40170.0   
          2014-05-01         2   3434                   40170.0   

                      houseprice_avg_2019  elec_consumption_2019  IMDRank  \
MSOA11CD  date                                                              
E02000002 2014-01-01             357206.0           12874.224118   6621.5   
          2014-02-01             357206.0           12874.224118   6621.5   
          2014-03-01             357206.0           12874.224118   6621.5   
          2014-04-01             357206.0           12874.224118   6621.5   
          2014-05-01             357206.0           12874.224118   6621.5   

                      IMDDecil                    RUC11  pv_count  \
MSOA11CD  date                                                      
E02000002 2014-01-01       2.0  Urban major conurbation       0.0   
          2014-02-01       2.0  Urban major conurbation       1.0   
          2014-03-01       2.0  Urban major conurbation       1.0   
          2014-04-01       2.0  Urban major conurbation       1.0   
          2014-05-01       2.0  Urban major conurbation       1.0   

                      public_charger_count  private_charger_count  n_months  \
MSOA11CD  date                                                                
E02000002 2014-01-01              2.363636                    0.5         0   
          2014-02-01              2.363636                    0.5         1   
          2014-03-01              2.363636                    0.5         2   
          2014-04-01              2.363636                    0.5         3   
          2014-05-01              2.363636                    0.5         4   

                      ev_price  
MSOA11CD  date                  
E02000002 2014-01-01  25278.29  
          2014-02-01  25278.29  
          2014-03-01  25278.29  
          2014-04-01  25278.29  
          2014-05-01  25278.29

In [40]:
#df_full.to_csv('data/msoa_2014_2021_no_onehotencode.csv')

## One hot encode categorical variables

The one categorical variable we have is the rural-ruban classification, of which there are 8 categories. We one hot encode and join with df_full.

In [41]:
pd.get_dummies(df_full['RUC11'])
df_full = pd.concat([df_full, pd.get_dummies(df_full['RUC11'])],axis=1)
df_full.drop(columns=['RUC11'],inplace=True)

In [44]:
print(df_full.shape)
df_full.head()

(505120, 20)


ev_count  index  total_annual_income_2019  \
MSOA11CD  date                                                    
E02000002 2014-01-01         2   3434                   40170.0   
          2014-02-01         2   3434                   40170.0   
          2014-03-01         2   3434                   40170.0   
          2014-04-01         2   3434                   40170.0   
          2014-05-01         2   3434                   40170.0   

                      houseprice_avg_2019  elec_consumption_2019  IMDRank  \
MSOA11CD  date                                                              
E02000002 2014-01-01             357206.0           12874.224118   6621.5   
          2014-02-01             357206.0           12874.224118   6621.5   
          2014-03-01             357206.0           12874.224118   6621.5   
          2014-04-01             357206.0           12874.224118   6621.5   
          2014-05-01             357206.0           12874.224118   6621.5   

                      IMDDecil  pv_count  public_charger_count  \
MSOA11CD  date                                                   
E02000002 2014-01-01       2.0       0.0              2.363636   
          2014-02-01       2.0       1.0              2.363636   
          2014-03-01       2.0       1.0              2.363636   
          2014-04-01       2.0       1.0              2.363636   
          2014-05-01       2.0       1.0              2.363636   

                      private_charger_count  n_months  ev_price  \
MSOA11CD  date                                                    
E02000002 2014-01-01                    0.5         0  25278.29   
          2014-02-01                    0.5         1  25278.29   
          2014-03-01                    0.5         2  25278.29   
          2014-04-01                    0.5         3  25278.29   
          2014-05-01                    0.5         4  25278.29   

                      Rural town and fringe  \
MSOA11CD  date                                
E02000002 2014-01-01                      0   
          2014-02-01                      0   
          2014-03-01                      0   
          2014-04-01                      0   
          2014-05-01                      0   

                      Rural town and fringe in a sparse setting  \
MSOA11CD  date                                                    
E02000002 2014-01-01                                          0   
          2014-02-01                                          0   
          2014-03-01                                          0   
          2014-04-01                                          0   
          2014-05-01                                          0   

                      Rural village and dispersed  \
MSOA11CD  date                                      
E02000002 2014-01-01                            0   
          2014-02-01                            0   
          2014-03-01                            0   
          2014-04-01                            0   
          2014-05-01                            0   

                      Rural village and dispersed in a sparse setting  \
MSOA11CD  date                                                          
E02000002 2014-01-01                                                0   
          2014-02-01                                                0   
          2014-03-01                                                0   
          2014-04-01                                                0   
          2014-05-01                                                0   

                      Urban city and town  \
MSOA11CD  date                              
E02000002 2014-01-01                    0   
          2014-02-01                    0   
          2014-03-01                    0   
          2014-04-01                    0   
          2014-05-01                    0   

                      Urban city and town in a sparse setting  \
MSOA11CD  date

In [45]:
#df_full.to_csv('data/msoa_full_updated_ev_count_onehotencode.csv')